In [63]:
%matplotlib inline
from torchvision.transforms import v2
import matplotlib.pyplot as plt
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
%load_ext autoreload
%autoreload 2
from preprocess.preprocess_data import preprocess_hugging_face_image

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [64]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


In [65]:
transforms = v2.Compose([
    v2.Resize(size=(256,256)),
    v2.PILToTensor(),
    v2.ToDtype(torch.float32),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [66]:
train_data_loader,test_data_loader = preprocess_hugging_face_image("microsoft/cats_vs_dogs",transforms,2)

In [67]:
for batch,data in enumerate(train_data_loader):
    print(batch)
    print(data['image'])
    print(data['labels'])
    break


0
tensor([[ 338.4934,  329.7598,  373.4279,  ...,  167.0845,  171.5289,
          175.9733],
        [ 779.5415,  775.1747,  779.5415,  ...,  593.7511,  589.3066,
          584.8622],
        [ 277.3581,  281.7249,  290.4585,  ...,  251.5289,  273.7511,
          269.3067],
        ...,
        [  45.9170,   50.2838,   50.2838,  ..., 1051.5289, 1042.6400,
         1038.1956],
        [ 299.1921,  281.7249,  272.9913,  ..., 1131.5289, 1131.5289,
         1131.5289],
        [ 722.7729,  727.1397,  731.5065,  ...,  131.5289,  127.0844,
          127.0844]])
tensor([1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1,
        1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0,
        1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
        0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0,
        1, 1, 1, 0])


In [53]:
from torch import nn
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(196608, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [57]:
def train_loop(dataloader, model, loss_fn, optimizer, batch_size):
    size = len(dataloader.dataset)
    model.train()
    for batch,data in enumerate(dataloader):
        X = data['image']
        y = data['labels']
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [55]:
def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    with torch.no_grad():
        for batch in dataloader:
            X = batch['image']
            y = batch['labels']
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [68]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_data_loader, model, loss_fn, optimizer,64)
    test_loop(test_data_loader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 6.312877  [  100/21069]
loss: 6.221664  [ 6500/21069]
loss: 6.313904  [12900/21069]
Test Error: 
 Accuracy: 48.4%, Avg loss: 6.278581 

Epoch 2
-------------------------------
loss: 6.273675  [  100/21069]
